In [1]:
site_code= '4832'

In [2]:
import pandas as pd

df_menu = pd.read_excel('./part_out.xlsx', header=0)
list_info = df_menu['Info'].to_list()

print(len(list_info))
list_info

193


['https://www.rockauto.com/en/moreinfo.php?pk=817402',
 'https://www.rockauto.com/en/moreinfo.php?pk=1319955',
 'https://www.rockauto.com/en/moreinfo.php?pk=1352614',
 'https://www.rockauto.com/en/moreinfo.php?pk=881754',
 'https://www.rockauto.com/en/moreinfo.php?pk=1002851',
 'https://www.rockauto.com/en/moreinfo.php?pk=1002394',
 'https://www.rockauto.com/en/moreinfo.php?pk=984822',
 'https://www.rockauto.com/en/moreinfo.php?pk=1354861',
 'https://www.rockauto.com/en/moreinfo.php?pk=1003015',
 'https://www.rockauto.com/en/moreinfo.php?pk=1514506',
 'https://www.rockauto.com/en/moreinfo.php?pk=1354123',
 'https://www.rockauto.com/en/moreinfo.php?pk=1353898',
 'https://www.rockauto.com/en/moreinfo.php?pk=10533372',
 'https://www.rockauto.com/en/moreinfo.php?pk=10617588',
 'https://www.rockauto.com/en/moreinfo.php?pk=1353910',
 'https://www.rockauto.com/en/moreinfo.php?pk=1352846',
 'https://www.rockauto.com/en/moreinfo.php?pk=657588',
 'https://www.rockauto.com/en/moreinfo.php?pk=1515

In [3]:
df = pd.DataFrame(columns=['Info',
                           'Src',
                           'OEM Number',
                           'Alternate Inventory Number',
                           'status'])

In [4]:
from urllib.parse import quote

In [5]:
i = 0

url = 'https://www.rockauto.com/en/moreinfo.php?pk=' + list_info[i].split('?')[1].split('pk=')[1].split('&')[0] + '&pt=' + site_code

url

'https://www.rockauto.com/en/moreinfo.php?pk=817402&pt=4832'

In [6]:
import requests

In [7]:
resp = requests.get(url).text

resp

'<!DOCTYPE HTML>\n<html class="no-js REPLACEHTMLCLASS" onclick="" lang="en">\n<head>\n<meta name="robots" content="all">\n<meta http-equiv="Content-Security-Policy" content="script-src \'self\' \'unsafe-inline\' \'unsafe-eval\' *.rockauto.com; base-uri \'self\' *.rockauto.com; ">\n<link rel="stylesheet" type="text/css" href="/jslib/combined/css/desktopmoreinfo.css?1685078658">\n\n<script>\nif(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};\n_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift()

In [8]:
from bs4 import BeautifulSoup

In [9]:
soup = BeautifulSoup(resp, 'lxml')

print(soup.prettify())

<!DOCTYPE HTML>
<html class="no-js REPLACEHTMLCLASS" lang="en" onclick="">
 <head>
  <meta content="all" name="robots"/>
  <meta content="script-src 'self' 'unsafe-inline' 'unsafe-eval' *.rockauto.com; base-uri 'self' *.rockauto.com; " http-equiv="Content-Security-Policy"/>
  <link href="/jslib/combined/css/desktopmoreinfo.css?1685078658" rel="stylesheet" type="text/css"/>
  <script>
   if(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};
_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift();for

In [10]:
with open('./oe.html', 'w', encoding='utf-8') as file:
    file.write(str(soup))

In [11]:
from lxml import etree

In [12]:
html = etree.HTML(str(soup))

list_src = html.xpath('//img[@alt="Part image"]/@src')
if len(list_src) == 0:
    list_src = ['']

list_src

['/info/154/CBS-1021_Front.jpg']

In [13]:
dict_alternate = {'alternate_inventory_number': '', 'oem_number': ''}
list_alternate = []
list_number = html.xpath('//section[@aria-label="Alternate/OEM Part Number(s)"]')
for number in list_number:
    list_alternate.append(''.join(number.xpath('./descendant-or-self::*/text()')))

list_alternate

['{Alternate Inventory Numbers: CBS-1021}',
 'OEM / Interchange Numbers: 8414006070, 8414033140']

In [17]:
for alternate in list_alternate:
    if '{Alternate Inventory Numbers: ' in alternate:
        dict_alternate['alternate_inventory_number'] = alternate.replace('{Alternate Inventory Numbers: ', '').replace('}', '').replace(', ', ';')
    elif 'OEM / Interchange Numbers: ' in alternate:
        dict_alternate['oem_number'] = alternate.replace('OEM / Interchange Numbers: ', '').replace(', ', ';')

dict_alternate

{'alternate_inventory_number': 'CBS-1021',
 'oem_number': '8414006070;8414033140'}

In [18]:
df_temp = pd.DataFrame({'Info': list_info[i],
                        'Src': list_src,
                        'OEM Number': [dict_alternate['oem_number']],
                        'Alternate Inventory Number': [dict_alternate['alternate_inventory_number']],
                        'status': ['ok']})

df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df

,Info,Src,OEM Number,Alternate Inventory Number,status
0,https://www.rockauto.com/en/moreinfo.php?pk=81...,/info/154/CBS-1021_Front.jpg,,,ok
1,https://www.rockauto.com/en/moreinfo.php?pk=81...,/info/154/CBS-1021_Front.jpg,8414006070;8414033140,CBS-1021,ok
